
![Dask Icon](images/dask_horizontal_black.gif "Dask Icon")
![Pandas Icon](images/pandas_logo.png "Pandas Icon")

# Gotcha's from Pandas to Dask

https://github.com/sephib/dask_pyconil2019

This notebook highlights some key differences when transfering code from `Pandas` to run in a `Dask` environment.  
Most issues have a link to the [Dask documentation](https://docs.dask.org/en/latest/) for additional information.

# Agenda  
1. Intro to `Dask` framework
2. Basic setup  `Client`
3. Dask.dataframe
4. Data manipulation
5. Read/Write files
6. Advanced `groupby`
7. Debugging


![Dask Icon](images/dask_horizontal_black.gif "Dask Icon")

Dask is a flexible library for parallel computing in Python.

![Dask Framework](images/dask_graph_outline.jpg)



Dask is composed of two parts:

1. *Dynamic task scheduling* optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
2. *“Big Data” collections* like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.

[link to documentation](https://docs.dask.org/en/latest/)

Dask emphasizes the following virtues:

* Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
* Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
* Native: Enables distributed computing in pure Python with access to the PyData stack.
* Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
* Scales up: Runs resiliently on clusters with 1000s of cores
* Scales down: Trivial to set up and run on a laptop in a single process
* Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans


See the [dask.distributed documentation (separate website)](https://distributed.dask.org/en/latest/) for more technical information on Dask’s distributed scheduler.

# [Why Dask?](https://docs.dask.org/en/latest/why.html)
* Scales from single comptuer out to clusters
* Familiar API
* Responsive feedback (live dashboard)
* ...

In [1]:
# since Dask is activly beeing developed - the current example is running with the below version
import dask
import dask.dataframe as dd
import pandas as pd
print(f'Dask versoin: {dask.__version__}')
print(f'Pandas versoin: {pd.__version__}')

Dask versoin: 1.2.2
Pandas versoin: 0.24.2


## Dask `Distributed` scheduler  

```python   
import dask.dataframe as dd  
from dask.distributed import Client 
client = Client()
df = dd.read_csv(...) # do something
``` 
vs  
\# When running code within a script use a `context manager`  
```python   
if __name__ == '__main__':
    with Client() as client:
        df = dd.read_csv(...) # do something
```


* see question in [stack overflow](https://stackoverflow.com/a/53520917/5817977)  
* In order to get url dashboard use [inner function ](https://github.com/dask/distributed/issues/2083#issue-337057906)  


## Start Dask Client for Dashboard
![Dask Dashboard](images/dask_dashboard.png)


In [2]:
from dask.distributed import Client 
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client()
client

Client Scheduler: tcp://127.0.0.1:36525 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 67.44 GB


Starting the Dask Client is optional.  In this example we are running on a `LocalCluster`, this  will also provide a dashboard which is useful to gain insight on the computation.  
For additional information on [Dask Client see documentation](https://docs.dask.org/en/latest/setup.html?highlight=client#setup)  

The link to the dashboard will become visible when you create a client (as shown below).  
When running in `Jupyter Lab` an [extenstion](https://github.com/dask/dask-labextension) can be installed to be able to view the various dashboard widgets. 

See [documentation for addtional cluster configuration](http://distributed.dask.org/en/latest/local-cluster.html)

# Create 2 DataFrames for comparison: 
* `Dask framework` is **lazy**  ![lazy python](images/Sleeping-snake.jpg)

In [3]:
ddf = dask.datasets.timeseries() #  Dask comes with builtin dataset samples, we will use this sample for our example. 
ddf

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In order to see the result we need to run [compute()](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.compute) 
 (or `head()` which runs under the hood compute()) )

In [4]:
ddf.compute()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307
2000-01-01 00:00:02,975,Ursula,-0.448956,0.006444
2000-01-01 00:00:03,926,Quinn,0.142141,0.717283
2000-01-01 00:00:04,957,Edith,-0.722394,-0.302324
2000-01-01 00:00:05,985,Victor,-0.199551,0.867684
2000-01-01 00:00:06,990,Yvonne,-0.755934,0.983617
2000-01-01 00:00:07,931,Michael,0.996341,-0.010357
2000-01-01 00:00:08,1047,Hannah,0.327315,-0.250445


## Pandas
In order to create a `Pandas` dataframe we can use the `compute()` 

In [5]:
pdf = ddf.compute()  
print(type(pdf))
pdf.head()

<class 'pandas.core.frame.DataFrame'>


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307
2000-01-01 00:00:02,975,Ursula,-0.448956,0.006444
2000-01-01 00:00:03,926,Quinn,0.142141,0.717283
2000-01-01 00:00:04,957,Edith,-0.722394,-0.302324


## Creating a `Dask dataframe` from `Pandas`

In order to utilize `Dask` capablities on an existing `Pandas dataframe` (pdf) we need to convert the `Pandas dataframe` into a `Dask dataframe` (ddf)  with the [from_pandas](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.from_pandas) method. 
You must supply the number of `partitions` or `chunksize` that will be used to generate the dask dataframe

In [6]:
ddf2 = dd.from_pandas(pdf, npartitions=10)
print(type(ddf2))

ddf2.head() 

<class 'dask.dataframe.core.DataFrame'>


/home/ds/.local/share/virtualenvs/dask_pyconil2019-9doxB0Ra/lib/python3.7/site-packages/distributed/worker.py:3101: UserWarning: Large object of size 10.46 MB detected in task graph: 
  (                       id      name         x     ...  4 columns], 5)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307
2000-01-01 00:00:02,975,Ursula,-0.448956,0.006444
2000-01-01 00:00:03,926,Quinn,0.142141,0.717283
2000-01-01 00:00:04,957,Edith,-0.722394,-0.302324


## Partitions in Dask Dataframes

Notice that when we created a `Dask dataframe` we needed to supply an argument of `npartitions`.  
The number of partitions will assist `Dask` on how it's going to parallelize the computation.  
Each partition is a *separate* dataframe. For additional information see [partition documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#partitions)  



Using `reset_index()` method we can examin the partitions:  

In [7]:
pdf2 = pdf.reset_index()
pdf2.iloc[0] # Only 1 row

timestamp    2000-01-01 00:00:00
id                          1001
name                       Quinn
x                      -0.731392
y                       0.190787
Name: 0, dtype: object

Now lets look at a `Dask` dataframe

In [8]:
ddf2 = ddf2.reset_index() 
ddf2.loc[0].compute()  # each partition has an index=0

,timestamp,id,name,x,y
0,2000-01-01,1001,Quinn,-0.731392,0.190787
0,2000-01-04,979,Zelda,0.538824,0.284485
0,2000-01-07,978,Oliver,-0.946787,-0.607089
0,2000-01-10,993,Ursula,-0.435962,-0.843300
0,2000-01-13,1035,Ursula,0.423358,0.085119
0,2000-01-16,987,Bob,-0.417411,0.518391
0,2000-01-19,952,Bob,-0.822654,-0.835621
0,2000-01-22,1020,Quinn,0.915768,0.402694
0,2000-01-25,1078,Quinn,-0.927491,-0.563731
0,2000-01-28,966,Bob,0.821327,-0.690675


## dataframe.shape  
since `Dask` is lazy we cannot get the full shape before running `len`

In [9]:
print(f'Pandas shape: {pdf.shape}')
print('---------------------------')
print(f'Dask lazy shape: {ddf.shape}') 

Pandas shape: (2592000, 4)
---------------------------
Dask lazy shape: (Delayed('int-67740847-014c-4860-b099-0e216f5176ac'), 4)


In [10]:
print(f'Dask computed shape: {len(ddf.index):,}')  # expensive

Dask computed shape: 2,592,000


Now that we have a `dask` (ddf) and a `pandas` (pdf) dataframe we can start to compair the interactions with them.

# Moving from Update to Insert/Delete


![inplaceTrue](images/inplace_true.png "inplace_true")


Dask does not update - thus there are no arguments such as `inplace=True` which exist in Pandas.  
For more detials see [issue#653 on github](https://github.com/dask/dask/issues/653)

### Rename Columns

In [11]:
# Pandas 
print(pdf.columns)
pdf.rename(columns={'id':'ID'}, inplace=True)
pdf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

In [12]:
# Dask - Error
# ddf.rename(columns={'id':'ID'}, inplace=True)
# ddf.columns

'''
---------------------------------------------------------------------------  
TypeError                                 Traceback (most recent call last)  
<ipython-input-12-3e70ff3a549e> in <module>  
      1 # Dask - Error  
----> 2 ddf.rename(columns={'id':'ID'}, inplace=True)  
      3 ddf.columns  
TypeError: rename() got an unexpected keyword argument 'inplace'  
'''

"\n---------------------------------------------------------------------------  \nTypeError                                 Traceback (most recent call last)  \n<ipython-input-12-3e70ff3a549e> in <module>  \n      1 # Dask - Error  \n----> 2 ddf.rename(columns={'id':'ID'}, inplace=True)  \n      3 ddf.columns  \nTypeError: rename() got an unexpected keyword argument 'inplace'  \n"

* using `inplace=True` is *not* considerd to be *best practice*. 

In [13]:
# Dask or Pandas
print(ddf.columns)
ddf = ddf.rename(columns={'id':'ID'})
ddf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

## Data manipulation 

### loc - Pandas

In [14]:
mask_cond = (pdf['x']>0.5) & (pdf['x']<0.8)
pdf.loc[mask_cond, ['y']] = pdf['y']* 100
pdf[mask_cond].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:13,969,Quinn,0.780977,-30.325775
2000-01-01 00:00:17,957,Zelda,0.663829,9.138369


In [15]:
# Error
# cond_dask = (ddf['x']>0.5) & (ddf['x']<0.8)
# ddf.loc[cond_dask, ['y']] = ddf['y']* 100

'''
> TypeError                                 Traceback (most recent call last)  
> <ipython-input-16-2bbb2ae570bd> in <module> 
>       2 # Error  
> ----> 3 ddf.loc[cond_dask, ['y']] = ddf['y']* 100  
> TypeError: '_LocIndexer' object does not support item assignment  
'''

"\n> TypeError                                 Traceback (most recent call last)  \n> <ipython-input-16-2bbb2ae570bd> in <module> \n>       2 # Error  \n> ----> 3 ddf.loc[cond_dask, ['y']] = ddf['y']* 100  \n> TypeError: '_LocIndexer' object does not support item assignment  \n"

### Dask - use mask/where

```python
# Pandas
mask_cond = (pdf['x']>0.5) & (pdf['x']<0.8)
pdf.loc[mask_cond, ['y']] = pdf['y']* 100
```

In [16]:
mask_cond = (ddf['x']>0.5) & (ddf['x']<0.8)

ddf['y'] = ddf['y'].mask(cond=mask_cond, other=ddf['y']* 100)
ddf[mask_cond].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:13,969,Quinn,0.780977,-30.325775
2000-01-01 00:00:17,957,Zelda,0.663829,9.138369


[dask mask documentation](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.mask)

## Meta argument

> `meta` is the prescription of the names/types of the computation output   
[see stack overflow answer](https://stackoverflow.com/questions/44432868/dask-dataframe-apply-meta)

![crystal python](images/crystalBallsnake.png "crystal snake")
Since `Dask` creates a DAG for the computation it requires to understand what are the outputs of each calculation (see [meta documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#metadata))

In [17]:
pdf['initials'] = pdf['name'].apply(lambda x: x[0]+x[1])
pdf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa


In [18]:
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1])
ddf.head(2)

/home/ds/.local/share/virtualenvs/dask_pyconil2019-9doxB0Ra/lib/python3.7/site-packages/dask/dataframe/core.py:2345: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('name', 'object'))

  warnings.warn(meta_warning(meta))


,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa


#### Introducing meta argument

In [19]:
# Describe the outcome type of the calculation
meta_cal = pd.Series(object, name='initials')
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1]
                                    , meta = meta_cal)
ddf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa


In [20]:
def func(row, col1, col2):
    if (row[col1]> 0):  return row[col1] * 1000  
    else:        return row[col2] * -1
ddf['z'] = ddf.apply(func, args=('x', 'y'), axis=1
                     , meta=('z', 'float'))
ddf.head(2)

,ID,name,x,y,initials,z
timestamp,,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu,-0.190787
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa,202.071256


### Map partitions
* We can supply an ad-hoc function to run on each partition using the [map_partitions](https://dask.readthedocs.io/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions) method.   
Mainly useful for functions that are not implemented in `Dask` or `Pandas` . 
* Finally we can return a new `dataframe` which needs to be described in the `meta` argument  
The function could also include arguments.

In [21]:
import numpy as np
def func2(df, coor_x, coor_y, drop_cols):
    df['dist'] =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                           +  (df[coor_y] - df[coor_y].shift())**2 )
    df = df.drop(drop_cols, axis=1)
    return df

ddf2 = ddf.map_partitions(func2
                          , coor_x='x'
                          , coor_y='y'
                          , drop_cols=['initials', 'z']
                          , meta=pd.DataFrame({'ID':'i8'
                                              , 'name':str
                                              , 'x':'f8'
                                              , 'y':'f8'                                              
                                              , 'dist':'f8'}, index=[0]))
ddf2.head()

,ID,name,x,y,dist
timestamp,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,NaN
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,1.267266
2000-01-01 00:00:02,975,Ursula,-0.448956,0.006444,0.936178
2000-01-01 00:00:03,926,Quinn,0.142141,0.717283,0.924493
2000-01-01 00:00:04,957,Edith,-0.722394,-0.302324,1.336794


### Convert index into DateTime column

In [22]:
# Only Pandas
pdf = pdf.assign(times=pd.to_datetime(pdf.index).time)
pdf.head(2)

,ID,name,x,y,initials,times
timestamp,,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu,00:00:00
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa,00:00:01


In [23]:
# ddf.assign(times= dd.to_datetime(ddf.index).dt.time)
# Dask or Pandas
ddf = ddf.assign(times=ddf.index.astype('M8[ns]'))
ddf['times'] = ddf['times'].dt.time
ddf =client.persist(ddf)
ddf.head(2)

,ID,name,x,y,initials,z,times
timestamp,,,,,,,
2000-01-01 00:00:00,1001,Quinn,-0.731392,0.190787,Qu,-0.190787,00:00:00
2000-01-01 00:00:01,1017,Xavier,0.202071,-0.666307,Xa,202.071256,00:00:01


## Drop NA on column

In [24]:
pdf = pdf.drop(labels=['initials'],axis=1)
ddf = ddf.drop(labels=['initials','z'],axis=1)

In [25]:
pdf = pdf.assign(colna = None)
print(f'pandas: {pdf.head(1)}')
ddf = ddf.assign(colna = None)
print(f'dask: {ddf.head(1)}')

pandas:               ID   name         x         y     times colna
timestamp                                                  
2000-01-01  1001  Quinn -0.731392  0.190787  00:00:00  None
dask:               ID   name         x         y     times colna
timestamp                                                  
2000-01-01  1001  Quinn -0.731392  0.190787  00:00:00  None


In odrer for `Dask` to drop a column with all `na` we need to assist the graph

In [26]:
pdf = pdf.dropna(axis=1, how='all')
print(f'pandas: {pdf.head(1)}')
# check if all values in column are Null - expensive
if ddf.colna.isnull().all() == True:   
    ddf = ddf.drop(labels=['colna'],axis=1)
print(f'dask: {ddf.compute().head(1)}')

pandas:               ID   name         x         y     times
timestamp                                            
2000-01-01  1001  Quinn -0.731392  0.190787  00:00:00
dask:               ID   name         x         y     times
timestamp                                            
2000-01-01  1001  Quinn -0.731392  0.190787  00:00:00


##  Reset Index

In [27]:
# Pandas
pdf = pdf.reset_index(drop=True)
pdf.head(1)

,ID,name,x,y,times
0,1001,Quinn,-0.731392,0.190787,00:00:00
1,1017,Xavier,0.202071,-0.666307,00:00:01


In [28]:
# Dask
ddf = ddf.reset_index()
ddf = ddf.drop(labels=['timestamp'], axis=1 )
ddf.head(1)

,ID,name,x,y,times
0,1001,Quinn,-0.731392,0.190787,00:00:00
1,1017,Xavier,0.202071,-0.666307,00:00:01


# Read / Save files

When working with `pandas` and `dask` preferable try and work with [parquet](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#store-data-in-apache-parquet-format).  
Even so when working with `Dask` - the files can be read with multiple workers .  
Most `kwargs` are applicable for reading and writing files [see documentaion](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.to_csv) (including the option for output file naming).  
e.g. 
ddf = dd.read_csv('data/pd2dd/ddf*.csv', compression='gzip', header=False).  

However some are not available such as  `nrows`.

## Save files

In [31]:
%%time
# Pandas
from pathlib import Path
output_file = 'pdf_single_file.csv'
output_dir = Path('data/')
output_dir.mkdir(parents=True, exist_ok=True)
pdf.to_csv(output_dir / output_file)

CPU times: user 15 s, sys: 518 ms, total: 15.5 s
Wall time: 15.2 s


In [32]:
list(Path(output_dir).glob('*.csv'))

[PosixPath('data/pdf_single_file.csv')]

`Dask`
Notice the '*' to allow for multiple file renaming. 



In [33]:
%%time
# Dask
output_dask_dir = Path('data/pd2dd/')
output_dir.mkdir(parents=True, exist_ok=True)
ddf.to_csv(f'{output_dask_dir}/ddf*.csv', index = False)

CPU times: user 760 ms, sys: 71.5 ms, total: 831 ms
Wall time: 6.32 s


To find the number of partitions which will determine the number of output files use [dask.dataframe.npartitions](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.npartitions)  

In [34]:
ddf.npartitions

30

In [35]:
list(Path(output_dask_dir).glob('*.csv'))

[PosixPath('data/pd2dd/ddf20.csv'),
 PosixPath('data/pd2dd/ddf27.csv'),
 PosixPath('data/pd2dd/ddf06.csv'),
 PosixPath('data/pd2dd/ddf22.csv'),
 PosixPath('data/pd2dd/ddf18.csv'),
 PosixPath('data/pd2dd/ddf16.csv'),
 PosixPath('data/pd2dd/ddf03.csv'),
 PosixPath('data/pd2dd/ddf07.csv'),
 PosixPath('data/pd2dd/ddf12.csv'),
 PosixPath('data/pd2dd/ddf25.csv'),
 PosixPath('data/pd2dd/ddf10.csv'),
 PosixPath('data/pd2dd/ddf29.csv'),
 PosixPath('data/pd2dd/ddf09.csv'),
 PosixPath('data/pd2dd/ddf28.csv'),
 PosixPath('data/pd2dd/ddf19.csv'),
 PosixPath('data/pd2dd/ddf23.csv'),
 PosixPath('data/pd2dd/ddf02.csv'),
 PosixPath('data/pd2dd/ddf24.csv'),
 PosixPath('data/pd2dd/ddf15.csv'),
 PosixPath('data/pd2dd/ddf21.csv'),
 PosixPath('data/pd2dd/ddf13.csv'),
 PosixPath('data/pd2dd/ddf26.csv'),
 PosixPath('data/pd2dd/ddf17.csv'),
 PosixPath('data/pd2dd/ddf14.csv'),
 PosixPath('data/pd2dd/ddf01.csv'),
 PosixPath('data/pd2dd/ddf04.csv'),
 PosixPath('data/pd2dd/ddf08.csv'),
 PosixPath('data/pd2dd/ddf00

To change the number of output files use [repartition](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.repartition) which is an expensive operation.

## Read files

For `pandas` it is possible to iterate and concat the files [see answer from stack overflow](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe).

In [36]:
%%time
# Pandas
dir_path = Path(r'data/pd2dd')
concat_df = pd.concat([pd.read_csv(f) for f in list(dir_path.glob('*.csv'))])
len(concat_df)

CPU times: user 2.33 s, sys: 116 ms, total: 2.45 s
Wall time: 2.35 s


In [37]:
%%time
# Dask
_ddf = dd.read_csv('data/pd2dd/ddf*.csv')
len(_ddf)

CPU times: user 381 ms, sys: 24.3 ms, total: 405 ms
Wall time: 1.08 s


 ## Consider using Persist
Since Dask is lazy - it may run the **entire** graph/DAG (again) even if it already run part of the calculation in a previous cell.  Thus use [persist](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#persist-intelligently) to keep the results in memory 
```python
ddf = client.persist(ddf)
```
This is different from Pandas which once a variable was created it will keep all data in memory.  
Additional information can be read in this [stackoverflow issue](https://stackoverflow.com/questions/45941528/how-to-efficiently-send-a-large-numpy-array-to-the-cluster-with-dask-array/45941529#45941529) or see an exampel in [this post](http://matthewrocklin.com/blog/work/2017/01/12/dask-dataframes)   
This concept should also  be used when running a code within a script (rather then a jupyter notebook) which incoperates loops within the code.

In [38]:
_ddf = dd.read_csv('data/pd2dd/ddf*.csv')
# do some filter
ddf = client.persist(_ddf)
# do some computations
ddf.head(2)

,ID,name,x,y,times
0,1001,Quinn,-0.731392,0.190787,00:00:00
1,1017,Xavier,0.202071,-0.666307,00:00:01


# Group By - custom aggregations
In addition to the [groupby notebook example](https://github.com/dask/dask-examples/blob/master/dataframes/02-groupby.ipynb)  - 
this is another example how to try to eliminate the use of `groupby.apply`  
In this example we are grouping by columns into unique list.

In [43]:
# prepare pandas dataframe
pdf = pdf.assign(time=pd.to_datetime(pdf.index).time)
pdf['seconds'] = pdf.Time.astype(str).str[-2:]
cols_for_demo =['name', 'ID','seconds']
pdf[cols_for_demo].head()

,name,ID,seconds
0,Quinn,1001,00
1,Xavier,1017,00
2,Ursula,975,00
3,Quinn,926,00
4,Edith,957,00


In [44]:
%%time
pdf_gb = pdf.groupby(pdf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply
               (lambda x: list(set(x.to_list()))) 
               for att_col_gr in gp_col]
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')      

CPU times: user 1.14 s, sys: 31.9 ms, total: 1.17 s
Wall time: 1.13 s


In [42]:
df_edge_att.head(2)

,Weight,ID,seconds
name,,,
Alice,99844,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[72, 44, 02, 76, 58, 50, 64, 95, 06, 17, 20, 4..."
Bob,99843,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[72, 44, 02, 76, 58, 50, 64, 95, 06, 17, 20, 4..."


In any case sometimes using Pandas is more efficiante (assuming that you can load all the data into the RAM).  
In this case Pandas is faster

In [45]:
def set_list_att(x: dd.Series):
        return list(set([item for item in x.values]))
ddf['seconds'] = ddf.times.astype(str).str[-2:]
ddf = client.persist(ddf)
ddf[cols_for_demo].head(2)

,name,ID,seconds
0,Quinn,1001,00
1,Xavier,1017,01


In [46]:
%%time
# Dask option1 using apply
# notice the meta argument in the apply function
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].apply(set_list_att
                ,meta=pd.Series(dtype='object', name=f'{att_col_gr}_att')) 
               for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
    df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)

CPU times: user 3.22 s, sys: 221 ms, total: 3.44 s
Wall time: 6.66 s


Using [dask custom aggregation](https://docs.dask.org/en/latest/dataframe-api.html?highlight=dropna#dask.dataframe.groupby.Aggregation) is consideribly better

In [47]:
# Dask
import itertools
custom_agg = dd.Aggregation(
    'custom_agg', 
    lambda s: s.apply(set), 
    lambda s: s.apply(lambda chunks: list(set(itertools.chain.from_iterable(chunks)))),)

In [48]:
%%time
# Dask option1 using apply
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].agg(custom_agg) for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)  

CPU times: user 435 ms, sys: 37.8 ms, total: 472 ms
Wall time: 1.28 s


In [ ]:
df_edge_att.head()

## [Debugging](https://docs.dask.org/en/latest/debugging.html)
Debugging may be challenging...
1. Run code without client 
2. Verify integraty of DAG
3. Use Dashboard profiler

## Corrupted DAG

In [49]:
# reset dataframe
ddf = dask.datasets.timeseries()
ddf.head(1)

,id,name,x,y
timestamp,,,,
2000-01-01,964,Zelda,-0.752955,0.438433


In [50]:
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2  
                     +  (df[coor_y] - df[coor_y].shift())^2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))

In [52]:
# returns an error because of ^2
ddf.head()

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

* Even if the function is currected the DAG is corrupted

In [53]:
# Still results with an error
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                     +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))
ddf.head(2)

TypeError: unsupported operand type(s) for ^: 'float' and 'bool'

Need to reset the dataframe

In [54]:
ddf = dask.datasets.timeseries()
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                     +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))
ddf.head(2)

,id,name,x,y,col
timestamp,,,,,
2000-01-01 00:00:00,1048,Laura,-0.341021,-0.734015,NaN
2000-01-01 00:00:01,1001,Edith,-0.750700,-0.276933,0.613808


# Summary
1. `Dask` is lazy but efficient (parallel computing)
2. Flexable environments - from single laptop to thousands of nodes (`client`)
3. Usefull when comming from a `Pandas` (especially with comparison to `pyspark`) 
4. Bonus - dashboard
5. But beware of:
  * missing functionalities from `Pandas` API
  * currupted DAGs

js.berry@gmail.com

https://github.com/sephib/dask_pyconil2019